In [1]:
import re
import requests
import pandas as pd
from tqdm.notebook import tqdm
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool, Lock
from lxml import etree, html as lhtml
from bs4 import BeautifulSoup

In [2]:
%%time

n_threads = 70
n_page_first, n_page_final = 1, 250

def get_relod_page(n_page):
    request = requests.get('https://shop.relod.ru/catalog-products/4577/?num=20&sort=PROPERTY_RATING&order=desc&PAGEN_1={}'.format(n_page))
    if not request.ok:
        print(n_page, file=sys.stderr)
    with lock:
        pbar.update(1)
    return request

with ThreadPool(n_threads) as pool, tqdm(total=n_page_final) as pbar:
    lock = pbar.get_lock()
    pages = pool.map(get_relod_page, range(n_page_first, n_page_final + 1, 1))

pool.join()


Wall time: 8.96 s


In [3]:
%%time
def get_links_from_catalog(i):
    page = BeautifulSoup(pages[i].text, 'lxml')
    result = [("https://shop.relod.ru" + k.get('href')) for k in page.find_all(class_='bxr-item-image-wrap')]
    return result

if __name__ == '__main__':
    #with Pool(processes=2) as pool:
       # pass
    links = []
    with tqdm(total=len(pages)) as pbar:
        for i in range(len(pages)): 
            links += get_links_from_catalog(i)
            pbar.update(1)
#pool.join()
print(len(links))


5000
Wall time: 3min 47s


In [4]:
%%time

def get_relod_book(link_to_book):
    request = requests.get(link_to_book)
    if not request.ok:
        print(link_to_book, file=sys.stderr)
    with lock:
        pbar.update(1)
    return request

with ThreadPool(n_threads) as pool, tqdm(total=(len(links))) as pbar:
    lock = pbar.get_lock()
    books = pool.map(get_relod_book, links)

pool.join()


Wall time: 3min 57s


In [ ]:
%%time

metki = ['sale', 'hit', 'new', 'ptv']

def get_book_info(i):
    soup = BeautifulSoup(books[i].text, 'lxml')
    book_info = {
        'url' : links[i], 
        'Название' : re.sub(r'\s+', ' ', soup.h1.text),
        'Иллюстрации':  [('https:' + k.get('data-src')) for k in soup.find_all(itemprop='image')],
        'Метки':  [k.text for k in [soup.find(class_=('bxr-marker-' + n)) for n in metki] if k],
        'Оценка': soup.find(itemprop='ratingValue').get('content') if soup.find(itemprop='ratingValue') else '',
        'Число голосов':  soup.find(itemprop='ratingCount').get('content') if soup.find(itemprop='ratingCount') else '',
        'Наличие': soup.find(itemprop='availability').text,
        'Цена': soup.find(itemprop='price').get('content'),
        'Цена (скидка)': '',
        'Описание': re.sub(r'\s+', ' ', soup.find('div', class_='bxr-detail').text),
        }
    
    try:
        if soup.find(class_="js-ptv-check"):
            book_info['Цена (скидка)'] = re.sub(r'[^\d.]', '', re.search(r'"PRICE" : \d+.\d+,', soup.text).group(0))
    except:
        pass
    rows = soup.find(class_="bxr-props-table").find_all('tr', recursive=False)
    for row in rows:
        cols = [row.find(class_="bxr-props-name").text, re.sub(r'(^\s|\s+$)','', re.sub(r'\(сайт издательства\)', '', row.find(class_="bxr-props-data").text))]
        book_info[cols[0]] = cols[1]

    return book_info

books_info = []
with tqdm(total=len(books)) as pbar:
    for i in range(len(books)): 
        books_info += get_book_info(i)
        pbar.update(1)
    

In [6]:
df = pd.DataFrame(books_info)
df.to_csv('books.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
print(links[3])

https://shop.relod.ru/catalog-products/glory/


In [ ]:
from multiprocessing import Pool
import time

def f(x):
    print("function")
    return x*x

if __name__ == '__main__':
    with Pool(processes=1) as pool:         # start 4 worker processes
        #result = pool.apply_async(f, (10,)) # evaluate "f(10)" asynchronously in a single process
        #print(result.get())        # prints "100" unless your computer is *very* slow

        print(pool.map(f, range(10)))       # prints "[0, 1, 4,..., 81]"

        it = pool.imap(f, range(10))
        print(next(it))                     # prints "0"
        print(next(it))                     # prints "1"
        print(it.next(timeout=1))           # prints "4" unless your computer is *very* slow

        result = pool.apply_async(time.sleep, (10,))
        print(result.get(timeout=1))        # raises multiprocessing.TimeoutErro